## Imports

In [1]:
import pandas as pd
import numpy as np
import openpyxl

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

## Functions for preprocessing data

In [2]:
def process_dataframe(input_df):
    # Create a copy of the input DataFrame
    processed_df = input_df.copy()

    # Generate new column names based on the first two rows
    column_names = [
        processed_df.iloc[1][col] if col in processed_df.columns[0:2]
        else processed_df.iloc[0][col] + "_" + processed_df.iloc[1][col]
        for col in processed_df.columns
    ]

    # Assign the new column names to the DataFrame
    processed_df.columns = column_names

    # Skip the first two rows
    processed_df = processed_df.iloc[2:]

    # Reset the index
    processed_df.reset_index(drop=True, inplace=True)

    # Keep only specific columns based on their names
    columns_to_keep = [
        col for col in processed_df.columns
        if col.endswith("Name") or col.endswith("smiles") or col.endswith("min") or col.endswith("max")
    ]
    processed_df = processed_df[columns_to_keep]

    return processed_df

def normalize_dataframe(dataframe, variance_threshold=0.0):
    # Create a copy of the input DataFrame
    normalized_df = dataframe.copy()

    # Extract columns to normalize
    columns_to_normalize = dataframe.columns[2:]

    # Apply Min-Max scaling to the specified columns
    scaler                              = MinMaxScaler()
    normalized_df[columns_to_normalize] = scaler.fit_transform(dataframe[columns_to_normalize])

    # Apply variance thresholding to select relevant columns
    selector         = VarianceThreshold(threshold=variance_threshold)
    selected_columns = selector.fit_transform(normalized_df[columns_to_normalize])
    
    # Combine the selected columns with the original non-normalized columns
    normalized_df = normalized_df.iloc[:, :2].join(pd.DataFrame(selected_columns, columns=columns_to_normalize[selector.get_support()]))

    return normalized_df

def correlated_pairs(dataframe, threshold=0.95):
    # Create a correlation matrix and extract upper triangular part
    df_corr = dataframe.corr().abs()
    upper = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))

    # Find pairs of highly correlated columns
    correlated_pairs = []
    for col1 in upper.columns:
        for col2 in upper.columns:
            if col1 != col2 and upper[col1][col2] > threshold:
                correlated_pairs.append((col1, col2))

    # Print the correlated pairs if found
    if correlated_pairs:
        print(f"Highly correlated column pairs with a correlation coefficient above {threshold}:")
        for col1, col2 in correlated_pairs:
            print(f"Correlated Pair: {col1} : {col2}")
    else:
        print("No highly correlated column pairs found.")

    return None

def drop_columns(dataframe, substrings):
    # Identify columns containing specified substrings and drop them
    columns_to_drop = [col for col in dataframe.columns if any(substring in col for substring in substrings)]
    return dataframe.drop(columns=columns_to_drop)

## Load dataset

In [3]:
# Load experimental data and computational (DFT) data
excel      = "raw_dataset"
expt_sheet = "exp_data" # the product numbering is random (not comparable to publication numbering!)
elec_sheet = "DFT_elec"
nuc_sheet  = "DFT_nuc"

expt_df = pd.read_excel(excel + ".xlsx", expt_sheet, header=1,    engine='openpyxl')
el_load = pd.read_excel(excel + ".xlsx", elec_sheet, header=None, engine='openpyxl')
nu_load = pd.read_excel(excel + ".xlsx", nuc_sheet,  header=None, engine='openpyxl')

elec = process_dataframe(el_load)
nuc  = process_dataframe(nu_load)

In [4]:
elec_comp_name = elec.loc[:,'Compound_Name'].to_numpy()
nuc_comp_name  = nuc.loc[:,'Compound_Name'].to_numpy()

In [5]:
# elec = elec.drop(columns='smiles')
elec = elec.drop(columns='Compound_Name')

# nuc = nuc.drop(columns='smiles')
nuc = nuc.drop(columns='Compound_Name')

## Data pre-processing

### Normalization

In [6]:
# Normalization of individual dataframes using the 'normalize_dataframe' function, 
# with a variance threshold of 0.0 for feature selection.

el_norm = normalize_dataframe(elec, variance_threshold=0.0)
nu_norm = normalize_dataframe(nuc, variance_threshold=0.0)

### Cleaning of nucleophile/electrophile dataframes

In [7]:
# Filter uninterpretable or repeatitive features
# for electrophiles
el_filter_1 = ['μ', 'η', 'ω', 'polar', 'Hirsh', 'C_2.5Å','C_3.5Å', 'C_4.0Å']
el_filtered = drop_columns(el_norm, el_filter_1)

# for nucleophiles
nu_filter_1 = ['μ', 'η', 'ω', 'polar', 'Hirsh', '2.0Å', '2.5Å', '3.5Å', '4.0Å', '5.0']
nu_filtered = drop_columns(nu_norm, nu_filter_1)

In [8]:
# Check and remove correlated pairs for electrophiles

correlated_pairs(el_filtered, threshold=0.95)

print('--------------------------------------')
el_filter_2 = ['e_x2', 'e_x6', 'e_x39', 'e_x42', 'e_x43', 'e_x46', 'e_x47', 'e_x51', 'e_x55', 'e_x59','e_x75', 
               'e_x86', 'e_x90', 'e_x94', 'e_x95', 'e_x98', 'e_x99', 'e_x103', 'e_x106', 'e_x111','e_x118', 
               'e_x122', 'e_x123']

el_clean = drop_columns(el_filtered, el_filter_2)
correlated_pairs(el_clean, threshold=0.95)

Highly correlated column pairs with a correlation coefficient above 0.95:
Correlated Pair: e_x3_HOMO_max : e_x2_HOMO_min
Correlated Pair: e_x7_LUMO_max : e_x6_LUMO_min
Correlated Pair: e_x39_SASA_sphericity_max : e_x38_SASA_sphericity_min
Correlated Pair: e_x42_SASA_surface_area(Å²)_min : e_x38_SASA_sphericity_min
Correlated Pair: e_x42_SASA_surface_area(Å²)_min : e_x39_SASA_sphericity_max
Correlated Pair: e_x43_SASA_surface_area(Å²)_max : e_x38_SASA_sphericity_min
Correlated Pair: e_x43_SASA_surface_area(Å²)_max : e_x39_SASA_sphericity_max
Correlated Pair: e_x43_SASA_surface_area(Å²)_max : e_x42_SASA_surface_area(Å²)_min
Correlated Pair: e_x46_SASA_volume(Å³)_min : e_x38_SASA_sphericity_min
Correlated Pair: e_x46_SASA_volume(Å³)_min : e_x39_SASA_sphericity_max
Correlated Pair: e_x46_SASA_volume(Å³)_min : e_x42_SASA_surface_area(Å²)_min
Correlated Pair: e_x46_SASA_volume(Å³)_min : e_x43_SASA_surface_area(Å²)_max
Correlated Pair: e_x47_SASA_volume(Å³)_max : e_x38_SASA_sphericity_min
Cor

In [9]:
# Check and remove correlated pairs for nucleophiles

correlated_pairs(nu_filtered, threshold=0.95)

print('--------------------------------------')
nu_filter_2 = ['n_x3', 'n_x7', 'n_x39', 'n_x43', 'n_x46', 'n_x47', 'n_x50', 'n_x55', 'n_x59', 'n_x63',
              'n_x67', 'n_x83', 'n_x98', 'n_x111', 'n_x122', 'n_x118', 'n_x119', 'n_x143', 'n_x110']

nu_clean = drop_columns(nu_filtered, nu_filter_2)
correlated_pairs(nu_clean, threshold=0.95)

Highly correlated column pairs with a correlation coefficient above 0.95:
Correlated Pair: n_x3_HOMO_max : n_x2_HOMO_min
Correlated Pair: n_x7_LUMO_max : n_x6_LUMO_min
Correlated Pair: n_x43_SASA_surface_area(Å²)_max : n_x42_SASA_surface_area(Å²)_min
Correlated Pair: n_x46_SASA_volume(Å³)_min : n_x42_SASA_surface_area(Å²)_min
Correlated Pair: n_x46_SASA_volume(Å³)_min : n_x43_SASA_surface_area(Å²)_max
Correlated Pair: n_x47_SASA_volume(Å³)_max : n_x42_SASA_surface_area(Å²)_min
Correlated Pair: n_x47_SASA_volume(Å³)_max : n_x43_SASA_surface_area(Å²)_max
Correlated Pair: n_x47_SASA_volume(Å³)_max : n_x46_SASA_volume(Å³)_min
Correlated Pair: n_x51_NBO_charge_C_max : n_x50_NBO_charge_C_min
Correlated Pair: n_x55_NBO_charge_S_max : n_x54_NBO_charge_S_min
Correlated Pair: n_x59_NMR_shift_C_max : n_x58_NMR_shift_C_min
Correlated Pair: n_x62_distance_S_C(Å)_min : n_x55_NBO_charge_S_max
Correlated Pair: n_x63_distance_S_C(Å)_max : n_x54_NBO_charge_S_min
Correlated Pair: n_x63_distance_S_C(Å)_ma

### Non-normalized dataframes

In [10]:
# Return individual dataframes to non-normalized values

common_el = elec.columns.intersection(el_clean.columns)
final_el  = elec[common_el]
common_nu = nuc.columns.intersection(nu_clean.columns)
final_nu  = nuc[common_nu]

## Prepare for combining both dataframes

In [11]:
# insert compound names in dataframes again
final_el.insert(0, "Compound_Name", elec_comp_name, True)
final_nu.insert(0, "Compound_Name", nuc_comp_name, True)

In [12]:
# pre setting up new dataframes for reactions
new_e_df = pd.DataFrame()
new_n_df = pd.DataFrame()

new_e_df['Elec'] = elec_comp_name
new_n_df['Nuc']  = nuc_comp_name

In [13]:
# generating all possible combinations of electrophiles and nucleophiles
combinatorial_list  = [(x, y) for x in elec_comp_name for y in nuc_comp_name]
df_combi            = pd.DataFrame(combinatorial_list, columns=['Elec', 'Nuc'])
df_combi['Product'] = ['P' + str(i + 1) for i in range(len(df_combi))]
df_combi            = df_combi[['Product', 'Elec', 'Nuc']]

## Combined reaction dataframe

In [14]:
# Create a copy of 'expt_df' to initialize 'result_df'
result_df = expt_df.copy()

# Iterate over rows in 'expt_df' to fill in corresponding values from 'final_el'
for index, row in expt_df.iterrows():
    electrophile = row["Elec"]
    compound_row = final_el[final_el["Compound_Name"] == electrophile]
    if not compound_row.empty:
        result_df.loc[index, compound_row.columns[2:]] = compound_row.values[0, 2:]

# Iterate over rows in 'expt_df' to fill in corresponding values from 'final_nu'
for index, row in expt_df.iterrows():
    nucleophile = row["Nuc"]
    compound_row = final_nu[final_nu["Compound_Name"] == nucleophile]
    if not compound_row.empty:
        result_df.loc[index, compound_row.columns[2:]] = compound_row.values[0, 2:]

display(result_df)

,Product,Elec,Nuc,Success,T,e_x7_LUMO_max,e_x30_dipole(Debye)_min,e_x31_dipole(Debye)_max,e_x34_volume(Bohr_radius³/mol)_min,e_x35_volume(Bohr_radius³/mol)_max,...,n_x99_Sterimol_B1_S_H(Å)_morfeus_max,n_x102_Sterimol_B5_C_S(Å)_morfeus_min,n_x103_Sterimol_B5_C_S(Å)_morfeus_max,n_x106_Sterimol_B5_S_H(Å)_morfeus_min,n_x107_Sterimol_B5_S_H(Å)_morfeus_max,n_x114_Sterimol_L_S_H(Å)_morfeus_min,n_x115_Sterimol_L_S_H(Å)_morfeus_max,n_x142_pyramidalization_Agranat-Radhakrishnan_S_min,n_x146_pyramidalization_Gavrish_S(°)_min,n_x147_pyramidalization_Gavrish_S(°)_max
0,P1,E20,N3,1,90,-0.05318,5.6631,5.6631,922.383,922.383,...,1.100000,3.366479,3.661217,9.463870,9.856770,3.395656,3.397128,1.999996,10.583668,10.586495
1,P10,E23,N32,0,90,-0.04544,4.9212,4.9212,1318.997,1318.997,...,1.301187,3.538301,8.369518,4.398437,9.946915,2.852354,7.599742,1.626221,10.486909,12.610136
2,P100,E74,N3,1,90,-0.10458,5.6686,5.6686,1398.703,1398.703,...,1.100000,3.366479,3.661217,9.463870,9.856770,3.395656,3.397128,1.999996,10.583668,10.586495
3,P101,E65,N3,0,90,-0.07080,0.1577,0.1577,1518.991,1518.991,...,1.100000,3.366479,3.661217,9.463870,9.856770,3.395656,3.397128,1.999996,10.583668,10.586495
4,P102,E50,N3,0,90,-0.04259,2.8892,2.8892,622.346,622.346,...,1.100000,3.366479,3.661217,9.463870,9.856770,3.395656,3.397128,1.999996,10.583668,10.586495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,P95,E13,N14,1,90,-0.08772,6.0306,6.0306,1691.782,1691.782,...,1.100000,3.372458,3.372458,8.205389,8.205389,3.397582,3.397582,2.000000,10.574030,10.574030
163,P96,E13,N32,0,90,-0.08772,6.0306,6.0306,1691.782,1691.782,...,1.301187,3.538301,8.369518,4.398437,9.946915,2.852354,7.599742,1.626221,10.486909,12.610136
164,P97,E13,N13,1,90,-0.08772,6.0306,6.0306,1691.782,1691.782,...,1.100000,3.290456,3.290456,6.088793,6.088793,3.399187,3.399187,2.000000,10.633117,10.633117
165,P98,E32,N3,1,90,-0.07637,3.5957,3.5957,1282.284,1282.284,...,1.100000,3.366479,3.661217,9.463870,9.856770,3.395656,3.397128,1.999996,10.583668,10.586495


## Saving combined dataframe

In [15]:
# saving final dataframe
result_df.to_excel("pre_processed_dataset.xlsx") 